In [1]:
import transformers
import torch, math, einops
from src import Tokenizer, linear_unidirectional_graph_maker
from torch import nn

from torch.distributions.categorical import Categorical

In [2]:
tokenizer = Tokenizer('gpt2')
pretrained = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

In [3]:
from src.GPT2 import GPT2, GPT2_Encoder, GPT2_LM_Head

encoder=GPT2_Encoder()
decoder=GPT2_LM_Head()

model=GPT2(encoder, decoder, tokenizer,dropout=0)

model.load_from_original(pretrained)

graph_maker=linear_unidirectional_graph_maker(1000)


In [35]:
def generate(starting_string, number_of_tokens):
    inp=starting_string
    print(starting_string,end='')
    for _ in range(number_of_tokens):
        inp=tokenizer(starting_string)
        edge_index=graph_maker(len(inp))
        out_p=pretrained(inp)[0][-1]
        out=model(inp,edge_index)[-1]

        assert torch.allclose(out,out_p,1e-3,1e-3)
        temperature=1/1
        probabilities = Categorical(logits=out/temperature)
        sample=probabilities.sample().item()
        print(tokenizer.decode(sample),end='')
        starting_string=starting_string+tokenizer.decode(sample)

    return starting_string

In [36]:
text="Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry. "
out = generate(text, 60)


Legolas and Gimli advanced on the orcs, raising their weapons with a harrowing war cry.   They took the bloody elves by force and took their Déjà Vu to invade.   At the fighting, they were turned against each other, with both orcs and deer fighting each other.   The two clans became friends, throughout their feud.
From his point of view, he could